## Importing Packages

In [1]:
import sys
## The utils.py directory
sys.path.insert(0, '/Bath University/Semester 2/H Driver/Research Project/HOPEFULLY FINAAAAAAAAAAAAAAAL')

import utils
import numpy as np
import pandas as pd

from collections import Counter

from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, roc_curve, auc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Using TensorFlow backend.


## Adjust the Screen Width

In [2]:
pd.options.display.max_colwidth = 100

## Fix the Seed

In [3]:
np.random.seed(0)

## Read the Powergrading Dataset

In [4]:
## The directory of the cleaned SciEntsBank data after the preprocessing
scientsbank_df = pd.read_csv('data/SciEntsBank/cleaned_scientsbank.csv', dtype = str)

## TF-IDF

In [5]:
texts = np.array(scientsbank_df['question'] + ',' + scientsbank_df['referenceAnswers'] + ',' + scientsbank_df['answer']).astype('U')

In [6]:
texts[0]

'use several method separate identify substance mock rock separate salt water,water evaporate leave salt,let sit dish day'

In [7]:
txt_tfidf = utils.TFIDF(texts)

## Data Splitting

In [8]:
y = scientsbank_df[['correct']]

In [9]:
y['correct'] = y['correct'].astype(str).astype(int)

C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
X_train, X_test, y_train, y_test = train_test_split(txt_tfidf, y, test_size = 0.2, random_state = 0)

## Count the Obervations

In [11]:
counter = Counter(y['correct'])

In [12]:
negative_observations = counter[0]

In [13]:
positive_observations = counter[1]

In [14]:
print('Negative Obervations:', negative_observations)
print('Positive Obervations:', positive_observations)

Negative Obervations: 6038
Positive Obervations: 4226


## Build the XGBoost Model

In [15]:
def build_model():
    model = XGBClassifier(learning_rate    = 0.1,
                          n_estimators     = 1000,
                          max_depth        = 9,
                          min_child_weight = 1,
                          gamma            = 0.4,
                          subsample        = 0.9,
                          colsamole_bytree = 0.6,
                          reg_alpha        = 0.01,
                          eta              = 0.01,
                          objective        = 'binary:logistic',
                          nthread          = 4,
                          scale_pos_weight = negative_observations/positive_observations,
                          seed             = 27)
    return model

## Fit the Model

In [16]:
%%time

model = build_model()
model.fit(X_train, y_train)

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[09:57:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsamole_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Wall time: 25.5 s


XGBClassifier(base_score=0.5, booster='gbtree', colsamole_bytree=0.6,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              eta=0.01, gamma=0.4, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=9, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=4, nthread=4,
              num_parallel_tree=1, objective='binary:logistic', random_state=27,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1.4287742546142925,
              seed=27, subsample=0.9, tree_method='exact',
              validate_parameters=1, ...)

## Test the Model

In [17]:
y_pred = model.predict(X_test)

In [18]:
print('F1: {:.2f}{}'.format(f1_score(y_test, y_pred, average = 'weighted') * 100, '%'))

print('ROC: {:.2f}{}'.format(roc_auc_score(y_test, y_pred) * 100, '%'))

print('Precision: {:.2f}{}'.format(precision_score(y_test, y_pred, average = 'weighted') * 100, '%'))

print('Recall: {:.2f}{}'.format(recall_score(y_test, y_pred, average = 'weighted') * 100, '%'))

F1: 82.19%
ROC: 81.74%
Precision: 82.21%
Recall: 82.17%


In [19]:
# Print the confusion matrix
print('The Confusion Matrix')
print('--------------------')
print(metrics.confusion_matrix(y_test, y_pred))

print()

# Print the precision and recall, among other metrics
print('Metrics')
print('-------')
print(metrics.classification_report(y_test, y_pred, digits = 4))

The Confusion Matrix
--------------------
[[1010  188]
 [ 178  677]]

Metrics
-------
              precision    recall  f1-score   support

           0     0.8502    0.8431    0.8466      1198
           1     0.7827    0.7918    0.7872       855

    accuracy                         0.8217      2053
   macro avg     0.8164    0.8174    0.8169      2053
weighted avg     0.8221    0.8217    0.8219      2053



## Analyze the Results

In [20]:
y_test_arr = y_test.to_numpy().ravel()
results = pd.concat([pd.DataFrame(y_test.index.to_numpy()), pd.DataFrame(y_test_arr), pd.DataFrame(y_pred)], axis=1)
results.columns = ['Question_Number', 'Actual','Predicted']
incorrect_results = results[results['Predicted'] != results['Actual']]

In [21]:
incorrect_results.head()

,Question_Number,Actual,Predicted
1,5619,0,1
3,4433,0,1
24,272,0,1
38,3050,1,0
43,7508,0,1


In [22]:
answers     = []
questions   = []
ref_answers = []

for i in incorrect_results.Question_Number:
    answers.append(scientsbank_df.answer[i])
    questions.append(scientsbank_df.question[i])
    ref_answers.append(scientsbank_df.referenceAnswers[i])
    
incorrect_results['Answers'] = answers
incorrect_results['Questions'] = questions
incorrect_results['Ref_Answers'] = ref_answers

C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Lenovo\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [24]:
incorrect_results.head(5)

,Question_Number,Actual,Predicted,Answers,Questions,Ref_Answers
1,5619,0,1,think jam mixture,jim use solid water make mixture one one spoon solid 100 milliliter water clear nothing bottom m...,mixture clear
3,4433,0,1,need wire switch,denise make circuit light bulb run motor use special switch schematic diagram circuit switch ins...,one circuit complete time
24,272,0,1,would observe calcite look calcite rock think find put water,phil geologist want test calcite field bring acid vinegar describe phil test calcite would observe,put acid rock acid fizz phil would know rock calcite
38,3050,1,0,could practice start race,joe fast runner slow respond start gun could joe improve start time,joe could practice respond start gun improve joe start time
43,7508,0,1,would know food lizard want,elena male lizard live several year habitat provide know lizard territorial besides additional f...,elena include separate shelter lizard
